### Model Architecture

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from conversion import CSVToTensor

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_size = 9
        self.output_size = 9
        self.hidden_size = 2*27

        self.x0 = None
        self.x1 = None
        self.x2 = None
        self.x3 = None

        self.W1 = nn.Parameter(torch.randn(self.input_size, self.hidden_size))
        self.b1 = nn.Parameter(torch.randn(self.hidden_size))

        self.W2 = nn.Parameter(torch.randn(self.hidden_size, self.output_size))
        self.b2 = nn.Parameter(torch.randn(self.output_size))
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)
        self.crossloss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.train_data = None
        self.val_data = None
    
    def forward(self, x):
        self.x0 = x
        self.x1 = x @ self.W1 + self.b1
        self.x2 = self.relu(self.x1)
        self.x3 = self.x2 @ self.W2 + self.b2
        # x = self.dropout(x)
        
        return self.x3

    def load_data(self, file_path):
        dataloader = CSVToTensor(file_path)
        dataloader.create_all_tensor()
        dataset = dataloader.create_a_dataset()

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_data, self.val_data = random_split(dataset, [train_size, val_size])
        self.train_data = DataLoader(self.train_data, batch_size=16, shuffle=True)
        self.val_data = DataLoader(self.val_data, batch_size=16, shuffle=True)

    def train_model(self, epochs):
        self.to(self.device)
        self.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for src, trg in self.train_data:
                src = src.to(self.device)
                trg = trg.to(self.device)
                self.optimizer.zero_grad()
                output = self.forward(src)
                loss = self.crossloss(output, trg.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(self.train_data)
            print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}")
    

In [3]:
# if __name__ == '__main__':
#     model = Model()
#     # choose the dataset file path
#     model.load_data('./Datasets/tic_tac_toe_500_games.csv')
#     # choose the number of epochs
#     with torch.no_grad():
#         model.W1.copy_(torch.zeros(model.input_size, model.hidden_size))
#         model.b1.copy_(torch.ones(model.hidden_size))
#         model.W2.copy_(torch.zeros(model.hidden_size, model.output_size))
#         model.b2.copy_(torch.ones(model.output_size))
#     model.train_model(11)

In [4]:
M = Model()

### Neural Network Weight Calculation by Human Reflection

The following three cells demonstrate how a neural network's weights can be manually calculated to understand how updates affect the network. This process is aimed at providing educational insight into debugging neural networks.

We consider 6 possible input scenarios and manually calculate the appropriate weights to ensure correctness for each scenario without causing errors in others.

(Note: This is purely for educational purposes to understand debugging and does not alter the models themselves.)

---

In [5]:
mytensor1 = torch.tensor([[1,0,1],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor1 = mytensor1.reshape(1,9)
outtensor1 = torch.tensor([[0,2,0,0,0,0,0,0,0]], dtype=torch.float32)

print(mytensor1.view(3,3))
print("-----------------")

mytensor2 = torch.tensor([[1,1,0],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor2 = mytensor2.reshape(1,9)
outtensor2 = torch.tensor([[0,0,2,0,0,0,0,0,0]], dtype=torch.float32)

print(mytensor2.view(3,3))
print("-----------------")

mytensor3 = torch.tensor([[0,1,1],[0,0,0],[0,0,0]], dtype=torch.float32)
mytensor3 = mytensor3.reshape(1,9)
outtensor3 = torch.tensor([[2,0,0,0,0,0,0,0,0]], dtype=torch.float32)

print(mytensor3.view(3,3))
print("-----------------")

mytensor4 = torch.tensor([[1,0,0],[0,0,0],[1,0,0]], dtype=torch.float32)
mytensor4 = mytensor4.reshape(1,9)
outtensor4 = torch.tensor([[0,0,0,2,0,0,0,0,0]], dtype=torch.float32)

print(mytensor4.view(3,3))
print("-----------------")

mytensor5 = torch.tensor([[1,0,0],[1,0,0],[0,0,0]], dtype=torch.float32)
mytensor5 = mytensor5.reshape(1,9)
outtensor5 = torch.tensor([[0,0,0,0,0,0,2,0,0]], dtype=torch.float32)

print(mytensor5.view(3,3))
print("-----------------")

mytensor6 = torch.tensor([[0,0,0],[1,0,0],[1,0,0]], dtype=torch.float32)
mytensor6 = mytensor6.reshape(1,9)
outtensor6 = torch.tensor([[2,0,0,0,0,0,0,0,0]], dtype=torch.float32)

print(mytensor6.view(3,3))
print("-----------------")

tensor([[1., 0., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[1., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[0., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[1., 0., 0.],
        [0., 0., 0.],
        [1., 0., 0.]])
-----------------
tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 0.]])
-----------------
tensor([[0., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])
-----------------


In [6]:
with torch.no_grad():
    M.W1.copy_(torch.zeros(M.input_size, M.hidden_size))
    M.b1.copy_(torch.zeros(M.hidden_size))
    M.W2.copy_(torch.zeros(M.hidden_size, M.output_size))
    M.b2.copy_(torch.zeros(M.output_size))

    M.W1[0, 1] = 2
    M.W1[0, 0] = -2
    M.W1[0, 3] = 2

    M.W1[1,1] = -2
    M.W1[1, 2] = 2
    M.W1[1, 3] = -2
        
    M.W1[2, 0] = 2
    M.W1[2, 2] = -2
    M.W1[2 , 3] = - 2
    
    M.W1[3, 6] = 2
    M.W1[3 ,3] = -2
    M.W1[3, 1] = -2
    
    M.W1[6, 1] = -2
    M.W1[6, 6] = -2
    M.W1[6, 0] = 2
    M.W2[0:M.output_size , 0:M.output_size] = torch.eye(M.output_size)
print(M.W1)
# print(M.b1)
# print(M.W2)

Parameter containing:
tensor([[-2.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -2.,  2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  0., -2., -2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -2.,  0., -2.,  0.,  0.,  2.,  0.,  0.,  0.

In [7]:
output = M.forward(mytensor6)
x0 = M.x0
x1 = M.x1
x2 = M.x2
x3 = M.x3

print(x0)
print("----------------")
print(output)
print("----------------")
print("Excepted ")
print(outtensor2)
print("----------------")
print("Output argmax :")
output.argmax()
# loss = mytensor2 - outtensor2
# loss.sum()

tensor([[0., 0., 0., 1., 0., 0., 1., 0., 0.]])
----------------
tensor([[2., 0., 0., 0., 0., 0., 0., 0., 0.]], grad_fn=<AddBackward0>)
----------------
Excepted 
tensor([[0., 0., 2., 0., 0., 0., 0., 0., 0.]])
----------------
Output argmax :


tensor(0)

In [8]:
f1 = lambda in0, out0: out0-in0
# example of how work lambda function

### Neural Network Weight Calculation by Machine Reflection

The following three cells demonstrate how a neural network's weights can be calculated using machine reflection to understand how updates affect the network. This process is aimed at providing educational insight into debugging neural networks.

For each line of code, we'll reflect on the machine's operations to ensure correctness. We consider 6 possible input scenarios and calculate the appropriate weights to ensure they are correct for each scenario without causing errors in others.

The values are manually updated based on the results of the backpropagation provided by the model.

The dataset for training will be composed only of the 6 tensor that you seen above and will be in the ./Datasets/debug.csv

(Note: This is purely for educational purposes to understand debugging through machine reflection and does not alter the models themselves.)

---

In [427]:
#prepare the data for training,
# we don't split the data because we will evaluate manually
from torch.utils.data import Dataset

path = './Datasets/debug.csv'
loader = CSVToTensor(path)
loader.create_all_tensor()

class CustomDataset(Dataset):
    def __init__(self, input_tensor, output_tensor):
        self.input_tensor = input_tensor
        self.output_tensor = output_tensor

    def __len__(self):
        return len(self.input_tensor)

    def __getitem__(self, idx):
        input_data = self.input_tensor[idx]
        output_data = self.output_tensor[idx]
        return input_data, output_data

input_tensor = loader.game_tensor
output_tensor = loader.prediction_tensor

combined_dataset = CustomDataset(input_tensor, output_tensor)

combined_dataloader = DataLoader(combined_dataset, batch_size=6, shuffle=True)

In [ ]:
for src, trg in combined_dataloader:
    print("src : " , src, "\ntrg : " , trg)
    print("-----------------")

In [428]:
with torch.enable_grad():
    M.W1 = nn.Parameter(torch.randn(M.input_size, M.hidden_size))
    M.b1 = nn.Parameter(torch.randn(M.hidden_size))
    M.W2 = nn.Parameter(torch.randn(M.hidden_size, M.output_size))
    M.b2 = nn.Parameter(torch.randn(M.output_size))

#### Train the model and debug him

In [444]:
for _ in range (24):
    for src, trg in combined_dataloader:
        M.optimizer.zero_grad()

        output = M.forward(src)

        # loss = torch.pow(output - trg, 2).sum().sqrt()
        # loss = torch.nn.functional.mse_loss(output, trg)
        loss = M.crossloss(output, trg.argmax(dim=1))

        loss.backward()

        M.optimizer.step()

In [445]:
print ("\nloss :\t", loss.item())
print("learning rate : ", M.optimizer.param_groups[0]['lr'])
print ("source :\t", src)
print ("input :\t", M.x0)
print ("target :", trg)
print ("output :", output)
print ("\nW1 grad :\t", M.W1.grad)
print ('W1 grad max :\t', M.W1.grad.max().item())


loss :	 7.584810733795166
learning rate :  0.001
source :	 tensor([[1., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0.]])
input :	 tensor([[1., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0., 1., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 0., 0., 0.]])
target : tensor([[0., 0., 0., 2., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 0., 0., 0., 0., 0., 0.],
        [0., 2., 0., 0., 0., 0., 0., 0., 0.],
        [2., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 2., 0., 0.]])
output : tensor([[ -2.6347,  -4.4676, -12.7788,   2.0452, -14.1759,  -6.7255,   3.2349,
   

In [321]:
#Make a backup of the model parameters
W1_backup = M.W1
W2_backup = M.W2
b1_backup = M.b1
b2_backup = M.b2
loss_backup = loss

In [ ]:
print("loss backup : ", loss_backup)
print("----------------")
print(W1_backup)
print(b1_backup)
print("----------------")
print(W2_backup)
print(b2_backup)

In [277]:
# Restore the model parameters
M.W1 = W1_backup
M.W2 = W2_backup
M.b1 = b1_backup
M.b2 = b2_backup

#### Choose an input "in the dataset" for debug visualization

In [ ]:
i = 3

src_i = combined_dataloader.dataset.input_tensor[i]
trg_i = combined_dataloader.dataset.output_tensor[i]

print ("\ninput  :", src_i)
print ("target :", trg_i)

debug_output = M.forward(src_i)
print ("debug output :", debug_output)

# debug_loss = torch.pow(debug_output - trg_i, 2).sum().sqrt()
debug_loss = M.crossloss(debug_output, trg_i.argmax(dim=0))
print ("debug loss :\t", debug_loss)

debug_loss.backward()
print ("W1 grad :\t", M.W1.grad)



##### Example of how each function of learning can be simplified to undersand his meanning

In [ ]:
with torch.no_grad():

    M.W1.grad = M.W1.grad * 0
    newoutput = M.forward(src)
    print(newoutput)
    newloss = torch.abs(newoutput - loader.prediction_tensor[i]).sum()
    print(newloss)
    newloss = torch.pow(newoutput - loader.prediction_tensor[i], 2).sum().sqrt()
    print(newloss)
    newloss.backward()
    M.W1 -= (M.W1.grad) * 0.0001
    